# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
import os.path
from sklearn.model_selection import train_test_split

In [2]:
# diretory of the images
image_dir = Path('./dataset')

# Working on Dataset

In [3]:
# ** means any folder, * means any file with extension .jpg
filepaths = list(image_dir.glob(r'train/*.jpg'))

# Taking the name cat and dog from de images
labels = list(map(lambda x: os.path.split(x)[1][:3],filepaths)) 

# Creating the labels 
filepaths = pd.Series(filepaths, name = 'Filepaths')
labels = pd.Series(labels, name= 'Label')

image_df = pd.concat([filepaths,labels], axis = 1).astype(str)

In [4]:
train_df,  test_df = train_test_split(image_df, train_size = 0.95, shuffle = True,  random_state = 1)

# Load Image Data

In [5]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.2,
    validation_split = 0.1
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
)

In [6]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepaths',
    y_col='Label',
    target_size=(128,128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'training'
)

validation_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepaths',
    y_col='Label',
    target_size=(128,128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepaths',
    y_col='Label',
    target_size=(128,128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size = 32,
    shuffle = False
)

Found 21375 validated image filenames belonging to 2 classes.
Found 2375 validated image filenames belonging to 2 classes.
Found 1250 validated image filenames belonging to 2 classes.


In [7]:
train_images.class_indices

{'cat': 0, 'dog': 1}

# Training

In [ ]:
output_size = 2
hidden_layer_size = 256
inputs = (128,128,3)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (6,6), activation = 'relu', input_shape = inputs),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (4,4), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size/2, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

history = model.fit(
    train_images,
    validation_data = validation_images,
    epochs = 15,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor = 'val_loss',
            patience = 3
        )
    ]
)

Epoch 1/15
668/668 [==============================] - 408s 610ms/step - loss: 0.6995 - accuracy: 0.5853 - val_loss: 0.6679 - val_accuracy: 0.5827 - lr: 0.0010
Epoch 2/15
668/668 [==============================] - 422s 631ms/step - loss: 0.6182 - accuracy: 0.6622 - val_loss: 0.5965 - val_accuracy: 0.6825 - lr: 0.0010
Epoch 3/15
668/668 [==============================] - 434s 649ms/step - loss: 0.5729 - accuracy: 0.7012 - val_loss: 0.6716 - val_accuracy: 0.6042 - lr: 0.0010
Epoch 4/15
668/668 [==============================] - 422s 631ms/step - loss: 0.5414 - accuracy: 0.7276 - val_loss: 1.3725 - val_accuracy: 0.5844 - lr: 0.0010
Epoch 5/15
 19/668 [..............................] - ETA: 6:39 - loss: 0.4926 - accuracy: 0.7599